In [1]:
import urllib2
from bs4 import BeautifulSoup
import pandas as pd
from decimal import *
import numpy as np
import re
import unicodedata
import csv
from geopy.distance import vincenty

# Récupération des ports

## Source des données : http://www.vesseltracker.com/en/CoveredAreas.html

In [3]:
response = urllib2.urlopen('http://www.vesseltracker.com/en/CoveredAreas.html')
html = response.read()

In [4]:
page = BeautifulSoup(html)

/Users/inesfrebault/app/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [10]:
def recup_href(doc_html):
    with open('harbors.csv', 'w') as csvfile:
        fieldnames = ['Harbor', 'Latitude','Longitude']

        writer = csv.DictWriter(csvfile, delimiter=';',fieldnames=fieldnames)
        writer.writeheader()
        
        spamwriter = csv.writer(csvfile, delimiter=';',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        extract = []
        for i in range(13,2003):
            print i
            stringId = 'theLink_' + str(i)
            for link in doc_html.find_all(id=stringId):
                port = link.get('href')
                port = port.replace(" ","%20")
                string = 'http://www.vesseltracker.com' + port
                harbor = urllib2.urlopen(string)
                html2 = harbor.read()
                page2 = BeautifulSoup(html2)
                stringSplit=string.split('/')
            
                for a in page2.find_all('div',{'class' : 'info'}):
                    for b in a.find_all('div',{'class' : 'item'}):
                        if (b.get_text().find('Coordinates') > -1):
                            text=a.get_text()
                            text = text.encode('ascii', 'ignore')
                            Arraytext = text.split(":")
                            coord=Arraytext[1]
                            ArrayCoord = coord.split("'")
                            extract2 = []
                            #extract2.append(stringSplit[5])
                            #extract2.append(ArrayCoord[0])
                            #extract2.append(ArrayCoord[1])
                            spamwriter.writerow([stringSplit[5],ArrayCoord[0],ArrayCoord[1]])
            
        return extract

In [11]:
recup_href(page)

13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284

[]

# Conversion

In [ ]:
#N 41 19 
#E 019 27


In [36]:
def convert(string):
    stringSplitted = string.split(" ")
    result = float(stringSplitted[1]) + float(stringSplitted[2])/60
    if (stringSplitted[0] == "S" or stringSplitted[0] == "W"):
        result = -result
    return result

In [37]:
convert("N 41 19")

41.31666666666667

In [38]:
convert("E 019 27")

19.45

In [3]:
dfHarbor = pd.read_csv('./data/harbors.csv',sep=';')

In [4]:
dfHarbor.Longitude[0]

'E 019 27'

In [41]:
dfHarbor['Latitude2'] = dfHarbor.apply(lambda row: convert(row['Latitude']), axis=1)
dfHarbor['Longitude2'] = dfHarbor.apply(lambda row: convert(row['Longitude']), axis=1)

In [42]:
dfHarbor.to_csv("../data/harbors.csv",sep=';', encoding='utf-8')

# Distance

In [7]:
lat=-2.76805555556
lon=-80.435
threshold=40

#print shorlineAdenCurrent=shorlineAden[(shorlineAden.latitude<lat+threshold)

def get_distance(lat, lon,dataset):
    tab=[]
    tab2=[]
    pointAttack=(lat, lon)
    print pointAttack
    HarborCurrent=dataset[(dataset.Latitude2<lat+threshold) & (dataset.Latitude2>lat-threshold) & (dataset.Longitude2>lon-threshold) & (dataset.Longitude2<lon+threshold)]
    length=len(HarborCurrent)
    print 'len shorlineAdenCurrent : ' + str(length)

    for index in range(0,length):
        #print index
        pointCoast= (HarborCurrent.iloc[index]['Latitude2'], HarborCurrent.iloc[index]['Longitude2'])
        #print pointCoast
        distance =vincenty(pointCoast, pointAttack).kilometers
        #print 'distance current '+str(distance)
        tab.append(distance)
        tab2.append(HarborCurrent.iloc[index]['Harbor'])
        
    #print 'index : '+str(tab.index(min(tab)))
    print 'nom du port : '+tab2[tab.index(min(tab))]
    print(min(tab))
    
    return min(tab)

print 'distance : ' +str(get_distance(lat, lon,dfHarbor))

(-2.76805555556, -80.435)
len shorlineAdenCurrent : 230
nom du port : posorja
21.9041032832
distance : 21.9041032832


In [8]:
df = pd.read_csv('../data/donneesBrutesPirateriesLatLon.csv',sep=';')
df['distance au port'] = df.apply(lambda row: get_distance(row['latitude'],row['longitude'],dfHarbor), axis=1)

(-6.10722222222, 106.90416666700001)
len shorlineAdenCurrent : 321
nom du port : jakarta
0.922289146372
(-3.21666666667, 116.383333333)
len shorlineAdenCurrent : 387
nom du port : tanjungpemancingan
11.1145276452
(-5.51833333333, 105.3)
len shorlineAdenCurrent : 342
nom du port : bakauheni
62.9974821144
(-1.7, 116.65)
len shorlineAdenCurrent : 408
nom du port : balikpapan
57.2439389036
(14.570555555599999, 120.92333333299999)
len shorlineAdenCurrent : 418
nom du port : manila
5.69358081354
(-6.00138888889, 105.934722222)
len shorlineAdenCurrent : 326
nom du port : cigading
2.39069083364
(3.20277777778, 104.02916666700001)
len shorlineAdenCurrent : 441
nom du port : kuantan
109.911939954
(3.20277777778, 104.02916666700001)
len shorlineAdenCurrent : 441
nom du port : kuantan
109.911939954
(1.30111111111, 104.213333333)
len shorlineAdenCurrent : 430
nom du port : batuampar
28.0128164649
(1.28555555556, 104.168611111)
len shorlineAdenCurrent : 430
nom du port : batuampar
22.9120424394
(1.3

In [9]:
df.to_csv("../data/donneesBrutesPirateriesLatLon.csv",sep=';', encoding='utf-8')

# Distance sur Aden.csv